In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, mixture
import matplotlib.pyplot as plt
tags_word2vec = np.loadtxt("./data/tag_word2vec.dat")
tags_fasttext = np.loadtxt("./data/tags_vector.dat")

In [2]:
gm = mixture.GaussianMixture(n_components=250, covariance_type='full')

In [3]:
z_gm = gm.fit(tags_word2vec)

In [4]:
labels = z_gm.predict(tags_word2vec)

In [5]:
label_num = len(set(labels))
label_num

250

In [6]:
weights = np.zeros(250)
for label in labels:
    weights[label] += 1
for i in range(250):
    weights[i] /= 2000

In [7]:
overweight = []
for i in range(len(weights)):
    if weights[i] > 0.025: #要素が2000*0.025=50より多い
        print(i)
        overweight.append(i)

1
17
164


In [8]:
def cos_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

zero = np.zeros(200)
for i in range(2000):
    if labels[i] in overweight:
        tmp = 0
        for j in range(2000):
            if (np.array_equal(tags_word2vec[j], zero) == False
                and labels[j] not in overweight 
                and cos_similarity(tags_fasttext[i], tags_fasttext[j]) > tmp):
                tmp = cos_similarity(tags_fasttext[i], tags_fasttext[j])
                labels[i] = labels[j]

In [9]:
df = pd.read_csv("./data/2000tags.csv")
df2 = pd.DataFrame({
    "word2vec": list(tags_word2vec),
    "fasttext": list(tags_fasttext),
    "クラスタ番号": list(labels)
})
df3 = pd.concat([df.loc[:,["タグ","投稿数"]],df2], axis=1)

df3.set_index("タグ").to_csv("./data/2000tags_gmm250.csv")

In [10]:
hashtags = [[] for i in range(label_num)]
for j in range(2000):
    hashtags[df3.loc[j]["クラスタ番号"]].append(df3.loc[j]["タグ"])

In [11]:
df3 = pd.DataFrame({
    "クラスタ番号": list(range(label_num)),
    "タグ": hashtags
})

df3.set_index("クラスタ番号").to_csv("./data/tags_per_cluster_gmm250.csv")

In [12]:
with open("./data/tags_per_cluster_gmm250.dat","w") as f:
  for elem in hashtags:
    f.write(str(elem) + "\n")
f.close()